In [348]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from umap import UMAP
import sqlite3 as lite
import numpy as np
from umap import UMAP

In [2]:
con = lite.connect('./data/src/diamonds_train.db/diamonds_train.db')

In [3]:
cursor = con.cursor()

In [4]:
raw_df = pd.read_sql('''SELECT *
FROM diamonds_transactional dt JOIN diamonds_dimensions ddi 
		ON dt.index_id = ddi.index_id 
	JOIN diamonds_properties dp ON dp.index_id = dt.index_id 
	JOIN diamonds_cut dc ON dc.cut_id = dp.cut_id 
	JOIN diamonds_color dc2 ON dc2.color_id = dp.color_id 
	JOIN diamonds_clarity dc3 ON dc3.clarity_id = dp.clarity_id 
	JOIN diamonds_city dc4 ON dc4.city_id = dt.city_id;''', con )

In [5]:
raw_df

,index_id,price,city_id,carat,index_id,depth,table,x,y,z,...,color_id,clarity_id,cut_id,cut,color_id,color,clarity_id,clarity,city_id,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,de88c121a82a06352bf1aaceba20578356408a334ba046...,Premium,6da43b944e494e885e69af021f93c6d9331c78aa228084...,J,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,Dubai
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,505,89c7286890f7347ab235234e74d406596a127ae3679042...,0.32,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,Very Good,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,H,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,89c7286890f7347ab235234e74d406596a127ae3679042...,Kimberly
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2686,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,0.71,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,Fair,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,G,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...,VS1,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,Las Vegas
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,738,89c7286890f7347ab235234e74d406596a127ae3679042...,0.41,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,Good,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,D,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...,SI1,89c7286890f7347ab235234e74d406596a127ae3679042...,Kimberly
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4882,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,Ideal,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,G,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...,SI1,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,Dubai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,10070,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,1.34,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,Ideal,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,G,ef736c1f91cd1900c3d9cde925b1bf4f013adc0211a9ee...,VS1,ca3aa21a5b70c3e88cc6336682c8c7da928a0c66a5ead4...,Antwerp
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,12615,e9c722cbefc2f055ae60b4e2cbe73a2d99537eab0c37f3...,2.02,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,...,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,Good,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,F,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...,SI2,e9c722cbefc2f055ae60b4e2cbe73a2d99537eab0c37f3...,Madrid
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,5457,89c7286890f7347ab235234e74d406596a127ae3679042...,1.01,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,

In [6]:
raw_df.describe()

,price,carat,depth,table,x,y,z
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,3928.444469,0.797706,61.752841,57.446133,5.729392,5.732819,3.537154
std,3992.416147,0.475544,1.431725,2.233535,1.124453,1.146650,0.697062
min,326.000000,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000
25%,945.000000,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000
50%,2397.000000,0.700000,61.800000,57.000000,5.690000,5.710000,3.520000
75%,5331.000000,1.040000,62.500000,59.000000,6.540000,6.540000,4.035000
max,18823.000000,4.500000,79.000000,95.000000,10.230000,58.900000,8.060000


In [7]:
raw_df.columns

Index(['index_id', 'price', 'city_id', 'carat', 'index_id', 'depth', 'table',
       'x', 'y', 'z', 'index_id', 'cut_id', 'color_id', 'clarity_id', 'cut_id',
       'cut', 'color_id', 'color', 'clarity_id', 'clarity', 'city_id', 'city'],
      dtype='object')

In [426]:
clean_df = raw_df[ ['price', 'carat',  'depth', 'table','x', 'y', 'z','cut', 'color', 'clarity', 'city']]

In [427]:
clean_df.to_csv('./data/clean/clean.csv')
clean_df

,price,carat,depth,table,x,y,z,cut,color,clarity,city
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,Premium,J,VS2,Dubai
1,505,0.32,63.0,57.0,4.35,4.38,2.75,Very Good,H,VS2,Kimberly
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,Fair,G,VS1,Las Vegas
3,738,0.41,63.8,56.0,4.68,4.72,3.00,Good,D,SI1,Kimberly
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,Ideal,G,SI1,Dubai
...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,Ideal,G,VS1,Antwerp
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,Good,F,SI2,Madrid
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,Ideal,H,SI1,Kimberly
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,Ideal,J,VS1,Kimberly


In [428]:
def data_labeling(df, cols):
   dic = {}
   for col in cols:
      group_by = df[[col,'price']].groupby(col).median().sort_values('price')
      for key in group_by.index:
         dic[key] = group_by.loc[key,'price']/min(group_by['price'])
      print(dic)
      for key,value in dic.items():
         df.loc[df[col] == key,col] = value
      df[col] = df[col].astype(float)
   return dic


In [429]:
def data_labeling_test(df, cols, dic):
   for col in cols:
      for key,value in dic.items():
         df.loc[df[col] == key,col] = value
      df[col] = df[col].astype(float)


In [430]:
dic2 = {'Ideal': 5,
 'Very Good': 3,
 'Good': 2,
 'Premium': 4,
 'Fair': 1,
 'E': 6,
 'D': 7,
 'G': 4,
 'F': 5,
 'H': 3,
 'I': 2,
 'J': 1,
 'IF': 8,
 'VVS1': 7,
 'VVS2': 6,
 'VS1': 5,
 'VS2': 4,
 'SI1': 3,
 'I1': 1,
 'SI2': 2,
 'Paris': 1.0,
 'Luxembourg': 1.0017263703064307,
 'Tel Aviv': 1.0051791109192922,
 'Zurich': 1.0310746655157532,
 'London': 1.0332326283987916,
 'Antwerp': 1.0358221838584376,
 'Madrid': 1.0360379801467414,
 'Las Vegas': 1.0388433318946915,
 'Surat': 1.0401381096245144,
 'New York City': 1.048769961156668,
 'Amsterdam': 1.0548122572291756,
 'Kimberly': 1.0561070349589987,
 'Dubai': 1.0699179974104445}

In [431]:
#dic = data_labeling(clean_df, ['cut', 'color', 'clarity','city'])
data_labeling_test(clean_df, ['cut', 'color', 'clarity','city'],dic2)
#clean_df = pd.concat([clean_df, pd.get_dummies(clean_df[['cut', 'color', 'clarity','city']], drop_first=True)], axis = 1).drop(['cut', 'color', 'clarity','city'],axis = 1)


C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\1822649115.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\1822649115.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\1822649115.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [432]:
len(dic2.keys())

33

In [433]:
sum(clean_df.isna().any(axis=1))

0

In [434]:
def feature_xy(x,y):
    try:
        return x/y
    except:
        return 1
def feature_ad(carat,x,y,z,):
    try:
        return carat/(x*y*z)
    except:
        return 1


In [435]:
clean_df['x/y'] = clean_df.apply(lambda x : feature_xy(x['x'], x['y']), axis=1)
#clean_df['x/z'] = clean_df.apply(lambda x : feature_xy(x['x'], x['z']), axis=1)
clean_df['ad'] = clean_df.apply(lambda x : feature_ad(x['carat'],x['x'], x['y'], x['z']), axis=1)

C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\3719788852.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return x/y
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\2549142257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['x/y'] = clean_df.apply(lambda x : feature_xy(x['x'], x['y']), axis=1)
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\3719788852.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  return carat/(x*y*z)
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\2549142257.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [436]:
sum(clean_df['x/y'].isna())

4

In [437]:
clean_df

,price,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,ad
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.006139
1,505,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.006107
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.006259
3,738,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.006187
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.006056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.006052
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,0.006229
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.006159
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.006011


In [438]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    40455 non-null  int64  
 1   carat    40455 non-null  float64
 2   depth    40455 non-null  float64
 3   table    40455 non-null  float64
 4   x        40455 non-null  float64
 5   y        40455 non-null  float64
 6   z        40455 non-null  float64
 7   cut      40455 non-null  float64
 8   color    40455 non-null  float64
 9   clarity  40455 non-null  float64
 10  city     40455 non-null  float64
 11  x/y      40451 non-null  float64
 12  ad       40455 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 4.0 MB


In [439]:
clean_df.columns

Index(['price', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color',
       'clarity', 'city', 'x/y', 'ad'],
      dtype='object')

In [440]:
'''selected_features = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color',
       'clarity', 'city']#,'x/y']#, 'y/z', 'x/z', 'apparent_density']
target = 'price''''

SyntaxError: EOL while scanning string literal (3393724755.py, line 3)

In [441]:
'''def filter_transform(df,selected_features,target):
    data_df = df[selected_features]
    to_dummy = ['city']
    for feature in to_dummy:
        dummies_df = pd.get_dummies(data_df[feature], prefix=feature, drop_first=True)
        data_df = pd.concat([data_df, dummies_df],axis=1).drop(feature, axis=1)   
    data_df = pd.concat([data_df.select_dtypes(exclude=['object']), df[target]], axis=1).dropna()
    return data_df'''

"def filter_transform(df,selected_features,target):\n    data_df = df[selected_features]\n    to_dummy = ['city']\n    for feature in to_dummy:\n        dummies_df = pd.get_dummies(data_df[feature], prefix=feature, drop_first=True)\n        data_df = pd.concat([data_df, dummies_df],axis=1).drop(feature, axis=1)   \n    data_df = pd.concat([data_df.select_dtypes(exclude=['object']), df[target]], axis=1).dropna()\n    return data_df"

In [442]:
#clean_df = pd.concat([clean_df, pd.get_dummies(clean_df[['cut', 'color', 'clarity','city']], drop_first=True)], axis = 1).drop(['cut', 'color', 'clarity','city'],axis = 1)

In [443]:
clean_df

,price,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,ad
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.006139
1,505,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.006107
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.006259
3,738,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.006187
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.006056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.006052
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,0.006229
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.006159
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.006011


In [444]:
#data_df = filter_transform(clean_df,selected_features,target)

In [445]:
clean_df

,price,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,ad
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.006139
1,505,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.006107
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.006259
3,738,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.006187
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.006056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.006052
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,0.006229
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.006159
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.006011


In [446]:
X = clean_df.drop('price', axis=1)
y = clean_df['price']

In [447]:
reducer = UMAP(n_components=10)
scaler = StandardScaler()

In [448]:
'''
X =X.fillna(0)
X = np.nan_to_num(X.astype(np.float32))
X = scaler.fit_transform(X)
X = reducer.fit_transform(X)
X'''

'\nX =X.fillna(0)\nX = np.nan_to_num(X.astype(np.float32))\nX = scaler.fit_transform(X)\nX = reducer.fit_transform(X)\nX'

In [449]:
X

,carat,depth,table,x,y,z,cut,color,clarity,city,x/y,ad
0,1.21,62.4,58.0,6.83,6.79,4.25,4.0,1.0,4.0,1.069918,1.005891,0.006139
1,0.32,63.0,57.0,4.35,4.38,2.75,3.0,3.0,4.0,1.056107,0.993151,0.006107
2,0.71,65.5,55.0,5.62,5.53,3.65,1.0,4.0,5.0,1.038843,1.016275,0.006259
3,0.41,63.8,56.0,4.68,4.72,3.00,2.0,7.0,3.0,1.056107,0.991525,0.006187
4,1.02,60.5,59.0,6.55,6.51,3.95,5.0,4.0,3.0,1.069918,1.006144,0.006056
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.34,62.7,57.0,7.10,7.04,4.43,5.0,4.0,5.0,1.035822,1.008523,0.006052
40451,2.02,57.1,60.0,8.31,8.25,4.73,2.0,5.0,2.0,1.036038,1.007273,0.006229
40452,1.01,62.7,56.0,6.37,6.42,4.01,5.0,3.0,3.0,1.056107,0.992212,0.006159
40453,0.33,61.9,54.3,4.45,4.47,2.76,5.0,1.0,5.0,1.056107,0.995526,0.006011


In [450]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost regressor
xgb_model = XGBRegressor()

# Optimal gridsearch with data labeling
param_grid_dl = {
    'n_estimators': [225, 230, 235],
    'max_depth': [ 5, 6, 7],
    'learning_rate': [ 0.08, 0.07, 0.06],
    'missing': [np.inf]
}

param_grid_en = {
    'n_estimators': [100, 200, 300],
    'max_depth': [ 7, 9, 11],
    'learning_rate': [0.01, 0.1,0.2]
}


In [451]:

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_dl, cv=3, n_jobs=-1, verbose=2, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV instance to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)

# Evaluate the model on the test set using the best parameters
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score with best parameters: ", test_score)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters found:  {'learning_rate': 0.07, 'max_depth': 6, 'missing': inf, 'n_estimators': 230}
Best score found:  -527.556362377139
Test score with best parameters:  0.9905168297275211


In [171]:
rf_regressor = RandomForestRegressor()

# Define the parameter grid to search through
param_grid_rf = {
    'n_estimators': [200, 300, 400],
    'max_depth': [None],
    'min_samples_split': [1,2, 3],
    'min_samples_leaf': [1, 2,3]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV instance to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)

# Evaluate the model on the test set using the best parameters
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score with best parameters: ", test_score)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
27 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\sklearn\base.py", line 1344, in wrapper
    estimator._validate_params()
  File "c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_

Best parameters found:  {'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 400}
Best score found:  -1367.9882830193383
Test score with best parameters:  0.8885594834273086


In [175]:
param_grid_lr = {
    'penalty': ['l1', 'l2'],
    'C': [ 0.1, 1, 10,]
}
logistic = LogisticRegression()
grid_search = GridSearchCV(estimator=logistic, param_grid=param_grid_lr, cv=3, n_jobs=None, verbose=2, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV instance to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters found: ", best_params)
print("Best score found: ", best_score)

# Evaluate the model on the test set using the best parameters
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Test score with best parameters: ", test_score)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


c:\Users\rjcol\.conda\envs\streamlit_env\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[CV] END ..................................C=0.1, penalty=l1; total time=   0.0s
[CV] END ..................................C=0.1, penalty=l1; total time=   0.0s
[CV] END ..................................C=0.1, penalty=l1; total time=   0.0s


KeyboardInterrupt: 

In [460]:
test = pd.read_csv('./data/src/diamonds_test.csv')

In [461]:
test.describe()

,id,carat,depth,table,x,y,z
count,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000,13485.000000
mean,6742.000000,0.798642,61.739095,57.490337,5.736454,5.739648,3.543474
std,3892.928525,0.469399,1.435310,2.237109,1.113671,1.128507,0.731005
min,0.000000,0.200000,50.800000,51.000000,0.000000,0.000000,0.000000
25%,3371.000000,0.400000,61.000000,56.000000,4.730000,4.730000,2.920000
50%,6742.000000,0.700000,61.900000,57.000000,5.700000,5.720000,3.530000
75%,10113.000000,1.040000,62.500000,59.000000,6.530000,6.530000,4.040000
max,13484.000000,5.010000,79.000000,73.000000,10.740000,31.800000,31.800000


In [462]:
data_labeling_test(test, ['cut', 'color', 'clarity','city'],dic2)

In [463]:
test['x/y'] = test.apply(lambda x : feature_xy(x['x'], x['y']), axis=1)
#test['x/z'] = test.apply(lambda x : feature_xy(x['x'], x['z']), axis=1)
test['ad'] = test.apply(lambda x : feature_ad(x['carat'],x['x'], x['y'], x['z']), axis=1)

C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\3719788852.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return x/y
C:\Users\rjcol\AppData\Local\Temp\ipykernel_7960\3719788852.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  return carat/(x*y*z)


In [464]:
#test = pd.concat([test, pd.get_dummies(test['city'], drop_first=True)], axis = 1).drop('city',axis = 1)

In [465]:
test2 = test
test2['price'] = 0

In [466]:
#test2 = test2[clean_df.columns]

In [467]:
'''def filter_transform2(df,selected_features,target):
    data_df = df[selected_features]
    to_dummy = [feature for feature in selected_features if len(df[feature].unique()) < 15]
    for feature in to_dummy:
        dummies_df = pd.get_dummies(data_df[feature], prefix=feature, drop_first=True)
        data_df = pd.concat([data_df, dummies_df],axis=1).drop(feature, axis=1)   
    data_df = pd.concat([data_df.select_dtypes(exclude=['object']), df[target]], axis=1)
    return data_df'''

"def filter_transform2(df,selected_features,target):\n    data_df = df[selected_features]\n    to_dummy = [feature for feature in selected_features if len(df[feature].unique()) < 15]\n    for feature in to_dummy:\n        dummies_df = pd.get_dummies(data_df[feature], prefix=feature, drop_first=True)\n        data_df = pd.concat([data_df, dummies_df],axis=1).drop(feature, axis=1)   \n    data_df = pd.concat([data_df.select_dtypes(exclude=['object']), df[target]], axis=1)\n    return data_df"

In [468]:
#data_test = filter_transform2(test2,selected_features,target)

In [469]:
X_submit = test2.drop(['id','price'], axis=1)

In [470]:
X_submit = X_submit[X.columns]

In [471]:
['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color', 'clarity',
       'Antwerp', 'Dubai', 'Kimberly', 'Las Vegas', 'London', 'Luxembourg',
       'Madrid', 'New York City', 'Paris', 'Surat', 'Tel Aviv', 'Zurich']

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut',
 'color',
 'clarity',
 'Antwerp',
 'Dubai',
 'Kimberly',
 'Las Vegas',
 'London',
 'Luxembourg',
 'Madrid',
 'New York City',
 'Paris',
 'Surat',
 'Tel Aviv',
 'Zurich']

In [472]:
first_sub = best_model.predict(X_submit)

In [473]:
test['price'] = first_sub

In [474]:
test['price']

0        2897.338135
1        5809.521484
2        9659.022461
3        4078.541748
4        1621.781494
            ...     
13480    1717.911499
13481    2459.347656
13482    3145.926270
13483    2155.663086
13484     784.094727
Name: price, Length: 13485, dtype: float32

In [214]:
test.set_index('id')['price'].to_csv('./last_submission.csv')